In [1]:
! gdown --id 1skw4zDUHJ0844UjL9-jmnimrgnGz0MqT

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1skw4zDUHJ0844UjL9-jmnimrgnGz0MqT
To: /content/weather.zip
100% 148M/148M [00:01<00:00, 96.3MB/s]


In [2]:
!unzip /content/weather.zip

Archive:  /content/weather.zip
  inflating: test/Image_1.jpg        
  inflating: test/Image_10.jpg       
  inflating: test/Image_100.jpg      
  inflating: test/Image_101.jpg      
  inflating: test/Image_102.jpg      
  inflating: test/Image_103.jpg      
  inflating: test/Image_104.jpg      
  inflating: test/Image_105.jpg      
  inflating: test/Image_106.jpg      
  inflating: test/Image_107.jpg      
  inflating: test/Image_108.jpg      
  inflating: test/Image_109.jpg      
  inflating: test/Image_11.jpg       
  inflating: test/Image_110.jpg      
  inflating: test/Image_111.jpg      
  inflating: test/Image_112.jpg      
  inflating: test/Image_113.jpg      
  inflating: test/Image_114.jpg      
  inflating: test/Image_115.jpg      
  inflating: test/Image_116.jpg      
  inflating: test/Image_117.jpg      
  inflating: test/Image_118.jpg      
  inflating: test/Image_119.jpg      
  inflating: test/Image_12.jpg       
  inflating: test/Image_120.jpg      
  inflating: test/I

In [3]:
import pandas as pd 
import numpy as np
import tensorflow as tf 
import tensorflow_hub as hub
import os 
import cv2
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder
import matplotlib as plt

In [4]:
y_train= pd.read_csv("/content/Training_set.csv")

In [5]:
path = [[filename, "/content/train/" + filename] for filename in y_train['filename']]
train_images = pd.DataFrame(path, columns= ['filename', 'path'])
train_data = pd.merge(train_images, y_train, how = 'inner', on = 'filename')

In [6]:
LE = LabelEncoder()
train_data.label = LE.fit_transform(train_data.label)
train_data.label.unique()

array([4, 3, 0, 1, 2])

In [7]:
train_data_pr = [] 
for i in range(len(train_data)):
  BndW = cv2.imread(train_data.iloc[i][1])
  BndW_rs = cv2.resize(BndW, (224,224))
  train_data_pr.append([BndW_rs, train_data.iloc[i][2]])

In [8]:
x= []
y = []
for row in train_data_pr:
  x.append(row[0])
  y.append(row[1])
x = np.array(x)
y = np.array(y)

In [9]:
X_train, X_val, y_TRAIN, y_VAL = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [10]:
X_train = X_train/255
X_val = X_val/255
X_train.shape

(838, 224, 224, 3)

In [11]:
cnn = tf.keras.Sequential([
  hub.KerasLayer("https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/5", trainable = False, input_shape= (224,224, 3)),
  tf.keras.layers.Dense(5, activation='softmax')
])
cnn.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
cnn.fit(X_train, y_TRAIN, epochs = 4, batch_size=50, validation_data=(X_val, y_VAL))

Epoch 1/4
17/17 [==============================] - 55s 3s/step - loss: 1.0523 - accuracy: 0.6217 - val_loss: 0.4729 - val_accuracy: 0.8762
Epoch 2/4
17/17 [==============================] - 41s 2s/step - loss: 0.3385 - accuracy: 0.8962 - val_loss: 0.2876 - val_accuracy: 0.9143
Epoch 3/4
17/17 [==============================] - 43s 3s/step - loss: 0.2096 - accuracy: 0.9439 - val_loss: 0.2357 - val_accuracy: 0.9333
Epoch 4/4
17/17 [==============================] - 41s 2s/step - loss: 0.1614 - accuracy: 0.9535 - val_loss: 0.2196 - val_accuracy: 0.9429


In [17]:
y_test= pd.read_csv("/content/Testing_set.csv")
path_test = [[filename, "/content/test/" + filename] for filename in y_test['filename']]
test_images = pd.DataFrame(path_test, columns= ['filename', 'path'])
test_data = pd.merge(test_images, y_test, how = 'inner', on = 'filename')
test_data.head()

,filename,path
0,Image_1.jpg,/content/test/Image_1.jpg
1,Image_2.jpg,/content/test/Image_2.jpg
2,Image_3.jpg,/content/test/Image_3.jpg
3,Image_4.jpg,/content/test/Image_4.jpg
4,Image_5.jpg,/content/test/Image_5.jpg


In [13]:
test_data_pr = [] 
for i in range(len(test_data)):
  BndW = cv2.imread(test_data.iloc[i][1])
  BndW_rs = cv2.resize(BndW, (224,224))
  test_data_pr.append(BndW_rs)


In [14]:
test_data_pr = np.array(test_data_pr)
test_data_pr.shape


(450, 224, 224, 3)

In [18]:
test_data_pr = test_data_pr/255
pred = cnn.predict(test_data_pr)
print(pred)

[[0.12737817 0.2625926  0.08828165 0.0637746  0.45797294]
 [0.12738383 0.26258495 0.08828636 0.06377904 0.45796585]
 [0.12739255 0.26259568 0.08828816 0.06377693 0.4579467 ]
 ...
 [0.12741359 0.26259145 0.08830798 0.06378581 0.45790115]
 [0.1273741  0.26262036 0.08826192 0.06376576 0.45797795]
 [0.12740329 0.26255465 0.08831212 0.06378868 0.45794135]]


In [ ]:
prediction = []
for data in pred:
  prediction.append(np.argmax(data))
final_predictions = LE.inverse_transform(prediction)

print(final_predictions)